In [0]:
## 구글 드라이브 연동

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
### KorQuAD V1 Download
import urllib
import zipfile

trainURL = 'https://github.com/korquad/korquad.github.io/raw/master/dataset/KorQuAD_v1.0_train.json'
devURL = 'https://github.com/korquad/korquad.github.io/raw/master/dataset/KorQuAD_v1.0_dev.json'


def download(url, savedir, zipped=True):
  file_name = url.split('/')[-1]
  download_file = savedir + file_name
  urllib.request.urlretrieve(url, download_file)

  if zipped:
    zip = zipfile.ZipFile(download_file)
    zip.extractall(savedir)
    zip.close()

SAVE_DIR = '/content/gdrive/My Drive/ai-summit/model/v1/'

In [0]:
download(trainURL, SAVE_DIR, False)
download(devURL, SAVE_DIR, False)

In [0]:
!ls -l /content/gdrive/My\ Drive/ai-summit/model/v1

total 41415
-rw------- 1 root root  3881058 Nov 19 23:23 KorQuAD_v1.0_dev.json
-rw------- 1 root root 38527475 Nov 19 23:23 KorQuAD_v1.0_train.json


In [0]:
## KorQuAD V1 format 확인

import json

def showJson(f, showcnt=0):
  json_file = open(f, encoding='utf-8')
  json_data = json.load(json_file)
  cnt = len(json_data['data'])
  if showcnt >= cnt:
    showcnt = con
  print('context : %s' % json_data['data'][showcnt]['paragraphs'][0]['context'])
  for i in range(len(json_data['data'][showcnt]['paragraphs'][0]['qas'])):
    q = json_data['data'][showcnt]['paragraphs'][0]['qas'][i]['question']
    a = json_data['data'][showcnt]['paragraphs'][0]['qas'][i]['answers'][0]['text']
    print('question %d : %s' % (i, q))
    print('answer %d : %s' % (i, a))
  #print('context : %s' % jsonstr['paragraphs'][0]['qas'][0]['question'])
  #print(json.dumps(json_data['data'][showcnt], indent=2, ensure_ascii=False))

In [0]:
showJson(SAVE_DIR+'KorQuAD_v1.0_train.json', 25)

context : 리처드 스타키 경(Sir Richard Starkey, MBE, 1940년 7월 7일 ~ )은 링고 스타(Ringo Starr)로 알려진 영국의 드러머, 가수, 작곡가, 배우다. 비틀즈의 드러머로 활동해 세계적인 유명세를 획득했다. 대개 한 음반의 한 곡에서 리드 보컬로 참여했고, 그중에는 〈With a Little Help from My Friends〉, 〈Yellow Submarine〉, 〈Good Night〉, 그들이 커버한 〈Act Naturally〉 등이 있다. 비틀즈 노래 〈Don't Pass Me By〉, 〈Octopus's Garden〉를 썼고, 〈What Goes On〉과 〈Flying〉 등에서는 공동 작곡가로 기록되었다.
question 0 : 영국의 드러머, 가수, 작곡가, 배우로 링고 스타로 잘 알려진 영국의 기사는? 
answer 0 : 리처드 스타키 경
question 1 : 영국의 드러머, 가수, 작곡가, 배우로 기사 작위를 받은 링고 스타의 본명은? 
answer 1 : 리처드 스타키
question 2 : 링고스타로 알려진 영국의 드러머, 가수, 작곡가 겸 배우는?
answer 2 : 리처드 스타키 경
question 3 : 비틀즈의 드러머로 활동해 세계적인 유명세를 획득한 인물은?
answer 3 : 리처드 스타키 경
question 4 : 영국의 링고 스타로 알려진 가수, 작곡가, 배우 겸 드러머는?
answer 4 : 리처드 스타키 경
question 5 : 비틀즈의 드러머로 활동했고, 비틀즈 노래  를 작곡한 가수는?
answer 5 : 리처드 스타키 경


### TPU 설정

In [0]:
import json
import os
import pprint
import sys
import tensorflow as tf

TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

TPU address is grpc://10.97.127.194:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 718154316226341280),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12665548956996096690),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17312329218632940829),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10557073809880534290),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 5564637767990611696),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0

## Google Cloud Platform storage bucket 생성 

#### - https://console.cloud.google.com/storage/browser

##### - model, output 폴더 생성

In [0]:
### model copy (google drive -> google cloud)

project_id = 'adroit-outlet-94907'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://bert-nlp-raonbit/


In [0]:
bucket_name = 'bert-nlp-raonbit'
!gsutil -m cp -r /content/gdrive/My\ Drive/ai-summit/model/multi_cased_L-12_H-768_A-12 gs://{bucket_name}/model

Copying file:///content/gdrive/My Drive/ai-summit/model/multi_cased_L-12_H-768_A-12/vocab.txt [Content-Type=text/plain]...
Copying file:///content/gdrive/My Drive/ai-summit/model/multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///content/gdrive/My Driv

In [0]:
!gsutil ls gs://{bucket_name}/model/multi_cased_L-12_H-768_A-12

gs://bert-nlp-raonbit/model/multi_cased_L-12_H-768_A-12/bert_config.json
gs://bert-nlp-raonbit/model/multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://bert-nlp-raonbit/model/multi_cased_L-12_H-768_A-12/bert_model.ckpt.index
gs://bert-nlp-raonbit/model/multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://bert-nlp-raonbit/model/multi_cased_L-12_H-768_A-12/vocab.txt


In [0]:
## Training(TPU)
!python /content/gdrive/My\ Drive/ai-summit/run_squad.py \
--vocab_file=gs://{bucket_name}/model/multi_cased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=gs://{bucket_name}/model/multi_cased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=gs://{bucket_name}/model/multi_cased_L-12_H-768_A-12/bert_model.ckpt \
--do_train=True \
--train_file=/content/gdrive/My\ Drive/ai-summit/model/v1/KorQuAD_v1.0_train.json \
--do_predict=True \
--predict_file=/content/gdrive/My\ Drive/ai-summit/model/v1/KorQuAD_v1.0_dev.json \
--train_batch_size=16 \
--max_seq_length=256 \
--doc_stride=128 \
--output_dir=gs://{bucket_name}/output \
--use_tpu=True \
--tpu_name={TPU_ADDRESS} \
--do_lower_case=False




W1124 01:34:05.073830 139977440577408 module_wrapper.py:139] From /content/gdrive/My Drive/ai-summit/run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1124 01:34:05.074091 139977440577408 module_wrapper.py:139] From /content/gdrive/My Drive/ai-summit/run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1124 01:34:05.074253 139977440577408 module_wrapper.py:139] From /content/gdrive/My Drive/ai-summit/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1124 01:34:07.828071 139977440577408 module_wrapper.py:139] From /content/gdrive/My Drive/ai-summit/run_squad.py:1133: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfc

### trainied model 복사

In [0]:
!gsutil -m cp -r gs://{bucket_name}/output /content/gdrive/My\ Drive/ai-summit/model/
!ls -l /content/gdrive/My\ Drive/ai-summit/model/output

total 10649365
-rw------- 1 root root        275 Nov 24 02:09 checkpoint
-rw------- 1 root root   15617130 Nov 24 02:09 eval.tf_record
-rw------- 1 root root   16227351 Nov 24 02:09 events.out.tfevents.1574559550.f9ba2fee0881
-rw------- 1 root root   10518088 Nov 24 02:09 graph.pbtxt
-rw------- 1 root root 2129535008 Nov 24 02:13 model.ckpt-10000.data-00000-of-00001
-rw------- 1 root root       9039 Nov 24 02:09 model.ckpt-10000.index
-rw------- 1 root root    4468627 Nov 24 02:09 model.ckpt-10000.meta
-rw------- 1 root root 2129535008 Nov 24 02:15 model.ckpt-11000.data-00000-of-00001
-rw------- 1 root root       9040 Nov 24 02:09 model.ckpt-11000.index
-rw------- 1 root root    4468627 Nov 24 02:09 model.ckpt-11000.meta
-rw------- 1 root root 2129535008 Nov 24 02:16 model.ckpt-11326.data-00000-of-00001
-rw------- 1 root root       9027 Nov 24 02:09 model.ckpt-11326.index
-rw------- 1 root root    4468627 Nov 24 02:09 model.ckpt-11326.meta
-rw------- 1 root root 2129535008 Nov 24 02:18

### Evaluation script

In [0]:
import urllib.request

SAVE_DIR = '/content/gdrive/My Drive/ai-summit/'

def download(url, savedir=SAVE_DIR):
  file_name = url.split('/')[-1]
  download_file = savedir + file_name
  urllib.request.urlretrieve(url, download_file)

evalscript = 'https://korquad.github.io/dataset/evaluate-v1.0.py'
download(evalscript)

In [0]:
import json

with open('/content/gdrive/My Drive/ai-summit/model/output/predictions.json', encoding='utf-8') as f:
  json_data = json.load(f)
  cnt = 10
  for k, v in json_data.items():
    if cnt == 0:
      break
    print('%s : %s' % (k, v))
    cnt -= 1

6548850-0-0 : 1989년 2월 15일
6548850-0-1 : 임수경
6548853-0-0 : 1989년
6548853-0-1 : 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관
6548853-0-2 : 서울지방경찰청 공안분실
6332405-0-0 : 임종석
6332405-0-1 : 여의도 농민 폭력 시위를
6548850-1-0 : 허영
6548850-1-1 : 10차 개헌안 발표
6332405-1-0 : 제89조


In [0]:
!python /content/gdrive/My\ Drive/ai-summit/evaluate-v1.0.py \
/content/gdrive/My\ Drive/ai-summit/model/v1/KorQuAD_v1.0_dev.json \
/content/gdrive/My\ Drive/ai-summit/model/output/predictions.json

{"exact_match": 69.41461724974022, "f1": 89.13237258962984}


- EM : exact match
- F1 : 예측한 답변과 실제 답변 사이의 중첩 토큰 부분을 측정

### Error Analysis
- 실제 답변과 예측 답변 확인

- [Machine Learning Yearning](https://www.deeplearning.ai/machine-learning-yearning/)

In [0]:
import json

devfile = '/content/gdrive/My Drive/ai-summit/model/v1/KorQuAD_v1.0_dev.json'
json_file = open(devfile, encoding='utf-8')
json_data = json.load(json_file)
data = json_data['data']

def getQnA(predict_id, dataset = data):
  for data in dataset:
    paragraphs = data['paragraphs']
    for paragraph in paragraphs:
      qas = paragraph['qas']
      for q in qas:
        id = q['id']
        if id == predict_id:
          return paragraph['context'], q['question'], q['answers'][0]['text']  # context, question, answer


In [0]:
with open('/content/gdrive/My Drive/ai-summit/model/output/predictions.json', encoding='utf-8') as f:
  json_data = json.load(f)
  cnt = 20
  for k, v in json_data.items():
    if cnt == 0:
      break
    context, question, real_answer = getQnA(k)
    if v != real_answer:
      print(context)
      print('question : %s' % question)
      print('real answer : prediction answer')
      print('%s : %s' % (real_answer, v))
      print()
      cnt -= 1

1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.
question : 임종석을 검거한 장소는 경희대 내 어디인가?
real answer : prediction answer
학생회관 건물 계단 : 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관

1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임